In [7]:
from PIL import Image
import numpy as np

def analyze_image(image_path):
    """
    Analyzes an image and returns the percentages of purple, blue, green,
    and the remaining pixels (excluding black and white).

    Parameters:
        image_path (str): Path to the input PNG image.

    Returns:
        dict: A dictionary containing percentages of different color categories.
            Keys: 'purple', 'blue', 'green'
    """
    # Open the image
    image = Image.open(image_path)

    # Check if the image has an alpha channel (RGBA)
    if image.mode == 'RGBA':
        # Get pixel data
        rgba_values = list(image.getdata())

        # Convert the list to a NumPy array for easier processing
        rgba_array = np.array(rgba_values)

        # Define boolean masks for black and white pixels
        black_mask = np.all((rgba_array[:, :3] >= (0, 0, 0)) & (rgba_array[:, :3] <= (25, 25, 25)), axis=1)
        white_mask = np.all((rgba_array[:, :3] >= (200, 200, 200)) & (rgba_array[:, :3] <= (255, 255, 255)), axis=1)

        # Calculate the percentage of black and white pixels
        percentage_black = (np.sum(black_mask) / rgba_array.shape[0]) * 100
        percentage_white = (np.sum(white_mask) / rgba_array.shape[0]) * 100

        # Get RGB values for remaining pixels
        remaining_pixels = rgba_array[~(black_mask | white_mask), :3]

        # Classify pixels into additional color categories
        purple_mask = ((remaining_pixels[:, 0] / remaining_pixels[:, 2] >= 0.8) & 
                       (remaining_pixels[:, 0] / remaining_pixels[:, 2] <= 1.3) &
                       (remaining_pixels[:, 1] / remaining_pixels[:, 0] >= 0) & 
                       (remaining_pixels[:, 1] / remaining_pixels[:, 0] <= 0.2))

        blue_mask = ((remaining_pixels[:, 0] < 50) & 
                         (remaining_pixels[:, 1] < 50) & 
                         (remaining_pixels[:, 2] > 100))

        green_mask = ((remaining_pixels[:, 1] > 50) & ~purple_mask & ~blue_mask)

        # Calculate percentages for additional color categories
        percentage_purple = (np.sum(purple_mask) / remaining_pixels.shape[0]) * 100
        percentage_blue = (np.sum(blue_mask) / remaining_pixels.shape[0]) * 100
        percentage_green = (np.sum(green_mask) / remaining_pixels.shape[0]) * 100

        # Calculate the percentage of remaining pixels
        percentage_remaining = 100 - percentage_black - percentage_white

        # Close the image
        image.close()

        # Return the results
        result = {
            'purple': percentage_purple,
            'blue': percentage_blue,
            'green': percentage_green,
        }

        return result

    else:
        # Close the image
        image.close()
        print("The image does not have an alpha channel (RGBA).")
        return None



In [8]:
image_path = 'original.png'
result = analyze_image(image_path)

if result:
    # Print the results
    print(f"\nPercentage of Purple pixels: {result['purple']:.2f}%")
    print(f"Percentage of Blue pixels: {result['blue']:.2f}%")
    print(f"Percentage of Green pixels: {result['green']:.2f}%")



Percentage of Purple pixels: 34.02%
Percentage of Blue pixels: 0.25%
Percentage of Green pixels: 0.82%
